In [6]:
import pandas as pd
import matplotlib as pt
import alpaca_trade_api as tradeapi
import requests
import yfinance as yf
from datetime import datetime, timedelta

In [7]:
# Alpaca API config stuff

APCA_API_BASE_URL = 'https://paper-api.alpaca.markets'
APCA_MARKET_DATA = 'https://data.alpaca.markets/v1'
APCA_API_KEY = 'PKDQTHU4Z3R8ZTXFOGXN'
APCA_SECRET_KEY = 'FBE3WStDzqd6WFATRUVQbDLRlht0E6p7kO6gLOdB'

api = tradeapi.REST(APCA_API_KEY, APCA_SECRET_KEY, APCA_API_BASE_URL)

In [8]:
# IEX API config stuff

IEX_CLOUD_API_KEY = 'Tpk_12e6eee6ed7d4026a0a87dee063b86bd'
IEX_CLOUD_SECRET_KEY = 'Tsk_620c4eafd98f479f990e532c3a0b2cb0'
SANDBOX_BASE_URL = 'https://sandbox.iexapis.com/stable/'

In [9]:
# 50-day SMA

def fifty_day_sma(ticker):
    FIFTY = 50.
    curr_date = str(datetime.now()).split()[0]
    year_ago = str(datetime.now() - timedelta(days=365)).split()[0]
    ticker_info = yf.Ticker(ticker)
 
    one_year_price_history = ticker_info.history(start=year_ago, end=curr_date)
    fifty_day_moving_average = one_year_price_history.iloc[-50:]['Close'].sum() / FIFTY
    return fifty_day_moving_average

In [10]:
# 200-day SMA

def two_hundred_day_sma(ticker):
    TWO_HUNDRED = 200.
    curr_date = str(datetime.now()).split()[0]
    year_ago = str(datetime.now() - timedelta(days=365)).split()[0]
    ticker_info = yf.Ticker(ticker)

    one_year_price_history = ticker_info.history(start=year_ago, end=curr_date)
    two_hundred_day_moving_average = one_year_price_history.iloc[-200:]['Close'].sum() / TWO_HUNDRED
    return two_hundred_day_moving_average

In [11]:
# checks if there is a Golden Cross (50-day SMA crosses 200-day SMA)

def golden_cross(ticker):
    fifty_sma = fifty_day_sma(ticker)
    two_hundred_sma = two_hundred_day_sma(ticker)
    
    if fifty_sma > two_hundred_sma:
        return True
    return False

In [12]:
# test SMAs + Golden Cross

print(fifty_day_sma('MSFT'), two_hundred_day_sma('MSFT'), golden_cross('MSFT'))

214.37353790283203 197.5154337310791 True
